### Imports

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
import re
import requests
import subprocess
from bs4 import BeautifulSoup
from contextlib import suppress
import escritoras_latinas.data.load as load
import pandas as pd

### Load data

In [21]:
data_processed = load.data_processed
data_escritoras = load.data_escritoras
assets = load.assets

### Read data

In [22]:
# Read 'csv' file as dataframe
df = pd.read_csv(f'{data_escritoras}')

# Create a list from a column values
urls = df['Url'].tolist()

df.sample(1)

,Nombre,Url,País,Biografía
904,Raquel Verdesoto,https://es.wikipedia.org/wiki/Raquel_Verdesoto,Ecuador,Ana María Raquel Verdesoto de Romo Dávila (Amb...


### Web scrapping

In [23]:
# img = 'https://upload.wikimedia.org/wikipedia/commons/5/57/Cristina_Peri.jpg'
# filename = 'test.jpg'
# # Download image using 'wget' command
# cmd = ['wget', '-O', f'{filename}', img]
# # Run the command to download
# subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

In [25]:
# HTML parsing to download images from Wikipedia
for url in urls:
    response = requests.get(url)
    # Return content of the response
    html = response.text
    # Parse html
    soup = BeautifulSoup(html, 'html.parser')
    # Look for <h1> tag with 'firstHeading' id
    title = soup.find('h1', attrs = {'id': 'firstHeading'})
    # Extract text from tag
    text = title.text
    # Process text
    name = text.lower().replace(' ', '_')
    # Look for <a> tag with 'image' class
    imgs = soup.find('a', attrs = {'class':'image'})
    # Handle known exceptions
    with suppress(Exception):
        # Look for <img> tag
        for img in imgs.find_all('img'):
            # Process url
            img = img['src'].replace('//','https://')
            # Ignore url with 'svg' extension
            if re.findall(r'.*\.(svg).*', img):
                continue
            else:
                # Match extension from url    
                extension = re.findall(r'.*\.(jpg|JPG|jpeg|JPEG|png|PNG)$', img)
                # Create file name
                filename = name+'.'+extension[0]
                # Download image using 'wget' command
                cmd = ['wget', '-O', f'{assets}/imgs/{filename}', img]
                # Run the command to download
                subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

In [26]:
# Generate file names in a directory tree
path, dirs, files = next(os.walk(f'{assets}/imgs'))
# Count files on directory
print(len(files))

470
